<a href="https://colab.research.google.com/github/annie-lyu/word-embedding/blob/master/ldaseq_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from gensim import corpora
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
import numpy
from gensim.matutils import hellinger
import nltk


In [11]:
import glob
from google.colab import drive
drive.mount('/content/drive')
##folder lists
ls = !ls "/content/drive/My Drive/words/sample"
ls = str(ls).strip("'[")
ls = ls.strip("]'")
entries = ls.split("\\t")
entries

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['month1', 'month2', 'month3']

In [12]:
## preparation for text preprocessing
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [13]:
#loading corpus and preprocessing
corpus_list= []
for entry in entries:
    filepath = "/".join(["/content/drive/My Drive/words/sample", entry, "*.txt"])
    data_files = glob.glob(filepath)
    for data_file in data_files:
      text = open(data_file,'r').read().lower()
      corpus = [word for word in text.split() if not word in stop_words] ##revome stop words 
      corpus_list += [corpus]
len(corpus_list)





1323

In [14]:
#3 Merge corpus into one document
with open("/content/drive/My Drive/words/sample/BleiCorpus.txt", 'w') as File:
  for corpus in corpus_list:
    File.write(" ".join(corpus)+ "\n" )

In [15]:
## establish dictionary 
dictionary = corpora.Dictionary(corpus_list)
dictionary.save("/content/drive/My Drive/words/sample/word_dictionary")

## create vocabulary file
class MyCorpus(object):
    def __iter__(self):
        for line in open("/content/drive/My Drive/words/sample/BleiCorpus.txt"):
            yield dictionary.doc2bow(line.lower().split())

corpus_memory_friendly = MyCorpus()
corpus = [vector for vector in corpus_memory_friendly]
corpora.BleiCorpus.serialize('/content/drive/My Drive/words/sample/news_corpus', corpus)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [16]:
## loading and setting
try:
  dictionary = Dictionary.load("/content/drive/My Drive/words/sample/word_dictionary")
except FileNotFoundError as e:
  raise ValueError("SKIP: Please download the Corpus/news_dictionary dataset.")

corpus = bleicorpus.BleiCorpus("/content/drive/My Drive/words/sample/news_corpus")

time_slice = [438, 429, 456]


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=dictionary, time_slice=time_slice, num_topics=5)

312228